In [ ]:
# 预训练代码


from transformers import AutoTokenizer, AutoModelForCausalLM
from model.configuration_qwen_agentic import CompressorConfig, ConverterConfig, DecoderConfig
from model.modeling_qwen_agentic import Qwen3AgenticConfig, Qwen3AgenticModel, Qwen3Agentic

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
config = Qwen3AgenticConfig.from_pretrained("./model")
config

/home/slgx/miniconda3/envs/memagent/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Qwen3AgenticConfig {
  "architectures": [
    "Qwen3Agentic"
  ],
  "auto_map": {
    "AutoConfig": "configuration_qwen_agentic.Qwen3AgenticConfig",
    "AutoModel": "modeling_qwen_agentic.Qwen3Agentic",
    "AutoModelForCausalLM": "modeling_qwen_agentic.Qwen3Agentic"
  },
  "compressor_config": {
    "architectures": [
      "Qwen3ForCausalLM"
    ],
    "attention_bias": false,
    "attention_dropout": 0.0,
    "base_model": "/home/slgx/yuhong/base_model/Qwen/Qwen3-0.6B",
    "bos_token_id": 151643,
    "dtype": "bfloat16",
    "eos_token_id": 151645,
    "head_dim": 128,
    "hidden_act": "silu",
    "hidden_size": 1024,
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "layer_types": [
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention",
      "full_attention

In [2]:

if hasattr(config.compressor_config, 'base_model'):
    print(config.compressor_config.base_model)


/home/slgx/yuhong/base_model/Qwen/Qwen3-0.6B


In [3]:
model = Qwen3Agentic(config)
model

✅ 成功加载压缩器: 'Qwen/Qwen3-0.6B'
✅ 初始化转换器完成


Loading checkpoint shards: 100%|██████████| 8/8 [00:05<00:00,  1.55it/s]

✅ 成功加载解码器: 'Qwen/Qwen3-14B


Qwen3Agentic(
  (compressor): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qw

In [4]:
model.decoder

Qwen3Model(
  (embed_tokens): Embedding(151936, 5120)
  (layers): ModuleList(
    (0-39): 40 x Qwen3DecoderLayer(
      (self_attn): Qwen3Attention(
        (q_proj): Linear(in_features=5120, out_features=5120, bias=False)
        (k_proj): Linear(in_features=5120, out_features=1024, bias=False)
        (v_proj): Linear(in_features=5120, out_features=1024, bias=False)
        (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
        (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
        (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
      )
      (mlp): Qwen3MLP(
        (gate_proj): Linear(in_features=5120, out_features=17408, bias=False)
        (up_proj): Linear(in_features=5120, out_features=17408, bias=False)
        (down_proj): Linear(in_features=17408, out_features=5120, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): Qwen3RMSNorm((5120,), eps=1e-06)
      (post_attention_layernorm): Qwen3RMSNorm((5120,), eps=1e-06)
    )
  )
  (norm): Qwen3RMSNorm((

In [6]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-14B")

In [10]:
prompt = "Hello, who are you?"
message = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    message,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)
text

'<|im_start|>user\nHello, who are you?<|im_end|>\n<|im_start|>assistant\n'

In [12]:
model_input = tokenizer([text], return_tensors="pt").to('cuda:0')
generate_ids = model.decoder.generate(**model_input, max_new_tokens=25)

AttributeError: 'Qwen3Model' object has no attribute 'generate'